In [ ]:
#install packages
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 42.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.5 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
#model name
model_name = 'microsoft/phi-2'
#dataset
dataset_name = 'hieunguyenminh/roleplay'
new_model_name = 'fine_tuned'

In [ ]:
#loading data
dataset = load_dataset(dataset_name, split = 'train[0:1000]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5755 [00:00<?, ? examples/s]

In [ ]:
#load base model with 4 bit quantisation
bnb_config = BitsAndBytesConfig(
                                load_in_4bit = True,
                                bnb_4bit_quant_type = 'nf4',
                                bnb_4bit_compute_dtype = torch.bfloat16,
                                bnb_4bit_use_double_quant = False
)
#create model
model = AutoModelForCausalLM.from_pretrained(
                                             model_name,
                                             quantization_config = bnb_config,
                                             device_map = 'auto',
                                             trust_remote_code = True
)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#no need to return last layers q k v
model.config.use_cache = False
#use approximate computation of fc layers
model.config.pretraining_tp = 1

In [ ]:
#load tokenizer
tokeniser = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
#pad with eos token
tokeniser.pad_token = tokeniser.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#create a pipeline to test model
pipe_pre_sft = pipeline(
                        task = 'text-generation',
                        model = model,
                        tokenizer = tokeniser,
                        max_length = 200
                        )

In [ ]:
#just try out a typical prompt before sft
prompt = '''
            <|system|> Batman is the consummate mortal superhero. A man walking amongst beings of immense power.
            Yet, his skills and desire for justice earn him their respect and even fear.
            <|user|> Tell me about your skills and abilities.
            <|assistant|> ...
'''

In [ ]:
result = pipe_pre_sft(prompt)
print(result[0]['generated_text'].split('<|assistant|> ...')[1])

 
        """
        return f"Batman is the consummate mortal superhero. A man walking amongst beings of immense power. Yet, his skills and desire for justice earn him their respect and even fear."

    def get_villain_description(villain: str) -> str:
        """
        Given a villain name, returns a string describing the villain's appearance, personality, and motivations.

        Args:
        - villain: a string representing the name of the villain

        Returns:
        - a string describing the villain's appearance, personality, and motivations
        """
        return f"{villain} is a


In [ ]:
#prepare for qlora finetuning
model = prepare_model_for_kbit_training(model)
#create a lora config
lora_config = LoraConfig(
                         r = 16,
                         lora_alpha = 16,
                         lora_dropout = 0.05,
                         bias = 'none',
                         task_type = 'CAUSAL_LM',
                         target_modules = [
                             'q_proj',
                             'k_proj',
                             'v_proj',
                             'dense',
                             'fc1',
                             'fc2'
                         ]
)
model = get_peft_model(model, lora_config)

In [ ]:
#training parameters
training_arguments = TrainingArguments(
                                       output_dir = './phi-2',
                                       warmup_steps = 1,
                                       max_steps = 5,
                                       learning_rate = 1e-3,
                                       fp16 = True,
                                       per_device_train_batch_size = 1,
                                       gradient_accumulation_steps = 1,
                                       optim = 'paged_adamw_32bit',
                                       logging_steps = 1
)
#create a trainer variable
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = lora_config,
    max_seq_length = 2048,
    dataset_text_field = 'text',
    tokenizer = tokeniser,
    args = training_arguments,
    packing = False
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#actually train
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.113100
2,1.278600
3,1.247400
4,1.575900
5,1.163100


TrainOutput(global_step=5, training_loss=1.2755958795547486, metrics={'train_runtime': 14.788, 'train_samples_per_second': 0.338, 'train_steps_per_second': 0.338, 'total_flos': 44684608665600.0, 'train_loss': 1.2755958795547486, 'epoch': 0.005})

In [ ]:
pipe_after_fine_tuning = pipeline(
    task = 'text-generation',
    model = model,
    tokenizer = tokeniser,
    max_length = 200
    )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

In [ ]:
result = pipe_after_fine_tuning(prompt)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
print(result[0]['generated_text'].split('<|assistant|> ...')[1])

 
            <|user|> What do you think sets Batman apart from other superheroes?
            <|assistant|> Batman's skills and desire for justice earn him their respect and even fear.
            <|user|> How do you feel about your role as Batman?
            <|assistant|> I feel honored to be able to use my skills and abilities to fight for justice and protect the innocent.
            <|user|> What motivates you to continue fighting?
            <|assistant|> The desire to make a difference and protect those who cannot protect themselves.
            <|user|> What do
